# Steam Workshop Downloader

Make sure you have `steamctl` installed. Use `pip install steamctl` to install it.

Edit the `private` file:
- first line should contain 'api XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
- second line should contain 'domain https:/your-domain.com'

You can obtain a Steam API key [here](https://steamcommunity.com/login/home/?goto=%2Fdev%2Fapikey). You can use your Steam profile URL as domain.

In [9]:
import os
import requests
import shutil

from tqdm.notebook import tqdm, trange
from bs4 import BeautifulSoup

Settings

- `N_PAGES`: Integer, Number of pages to scrap
- `APP_ID`: Integer, Steam application's ID (Space Engineers' is 244850)
- `FILTER`: String, arbitrary filter
- `SORT`: String, one of 'trend', 'mostrecent', 'lastupdated', 'totaluniquesubscribers'

In [10]:
N_PAGES = 30 # number of pages to scrap
APP_ID = 244850  # the Space Engineers App ID
FILTER = 'spaceship'  # filter string
SORT = 'trend'  # sorting order

API_KEY = ''
DOMAIN = ''

Create a directory to place the downloaded content.

In [11]:
os.makedirs('downloads', exist_ok=True)

Update the API_KEY in `steamctl`

In [12]:
with open('private', 'r') as f:
    ls = f.readlines()
    API_KEY = ls[0].strip().split(' ')[1]
    DOMAIN = ls[1].strip().split(' ')[1]

res = os.popen(f'steamctl webapi set-key {API_KEY}').read()
if not res.startswith('Current'):
    print('Error setting API key; check console.')

Scrape Workshop page to get the entries IDs

In [13]:
file_ids = []

for n in trange(N_PAGES, desc='Scraping workshop pages'):
    page_url = f'https://steamcommunity.com/workshop/browse/?appid={APP_ID}&searchtext={FILTER}&browsesort={SORT}&section=home&actualsort={SORT}&p={n}'
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text)
    entries = soup.find_all('a', class_='ugc')

    for entry in entries:
        file_ids.append(entry.get('data-publishedfileid'))

Scraping workshop pages:   0%|          | 0/30 [00:00<?, ?it/s]

Download each entry in its own folder

In [14]:
for i in trange(len(file_ids), desc='Downloading content'):
    file_id = file_ids[i]
    os.makedirs(f'downloads/{file_id}', exist_ok=True)
    os.popen(f'cd downloads/{file_id} && steamctl workshop download --anonymous {file_id}').read()

Remove content that does not contain structure blueprints (i.e.: does not contain any `.sbc` file)

In [15]:
subdirs = os.listdir('downloads')

to_remove = []

for subdir in subdirs:
    files = os.listdir(os.path.join('downloads', subdir))
    if 'bp.sbc' not in files:
        to_remove.append(os.path.join('downloads', subdir))

for i in trange(len(to_remove), desc='Removing bad folders'):
    shutil.rmtree(to_remove[i])

Removing bad folders:   0%|          | 0/625 [00:00<?, ?it/s]

Rename all `bp.sbc` files in each valid directory with the directory name (ID) and move them to a separate folder.

In [16]:
subdirs = os.listdir('downloads')

os.makedirs('content', exist_ok=True)

for i in trange(len(subdirs), desc='Moving content and cleaning downloads'):
    content_dir = os.path.join('downloads', subdirs[i])
    shutil.move(os.path.join(content_dir, 'bp.sbc'),
                os.path.join('content', f'{subdirs[i]}.sbc'))
shutil.rmtree('downloads')

Moving content and cleaning downloads:   0%|          | 0/241 [00:00<?, ?it/s]